In [5]:
import sys,glob
sys.path.append('/home/simon/git/burns/code/')

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_dir = '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/'
blank_mz_tol = 5
mz_tol = 5

In [8]:
group1 = ['MSS1','MSS2','MSS3','MSS4','MSS5','MSS6','MSS13']
group2 = ['MSS10','MSS11','MSS14']
files1 = []
files2 = []
for g in group1:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files1.append((match,data_dir+bfilename))
for g in group2:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files2.append((match,data_dir + bfilename))

group1,_ = zip(*files1)
group2,_ = zip(*files2)

print len(group1),len(group2)
print group1
print group2

12 9
('/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS1a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS2a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS3a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS4a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a3.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a2.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS6a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a1.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a2.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a3.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a.mzML')
('/home/simon/Dropbox/BioResearch/Met

In [9]:
from load_di import load_di

# s = load_di(group1[0])
import glob
data = {}
for f in files1+files2:
    out_file = '../csv_files/' + f[0].split('/')[-1].split('.')[0]+'.csv'
#     data[f[0]] = load_di(f[0],blank_file = f[1],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    data[f[0]] = load_di(f[0],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)


In [38]:
normalised_data = {}
for f,spec in data.items():
    mz_list,i_list = zip(*spec)
    tot_intensity = sum(i_list)
    print tot_intensity
    new_intensity = [1000.0*i/tot_intensity for i in i_list]
    new_spec = zip(mz_list,new_intensity)
    normalised_data[f] = new_spec

324412649.465
436240818.691
380777529.477
288935937.354
400955613.605
806308809.869
762854905.605
301898846.857
289888093.164
306846745.561
464233290.262
292485624.916
2107366630.76
320459909.518
450357167.426
277990991.938
307305363.902
476529778.904
506194463.586
462673639.387
290878928.832
414260544.383


Do some of Greedy matching

In [11]:
import numpy as np
from matching import Greedy
g = Greedy()
mp= g.process(data,mz_tol)
from scipy.stats import ttest_ind
small = 0.001
test_results = []
for ps in mp:
    g1 = []
    g2 = []
    for p in ps.peaks:
        filename = p[2]
        if filename in group1:
            g1.append(p[1])
        else:
            g2.append(p[1])
    if len(g1) <2 and len(g2) < 2:
        continue
    else:
        for i in range(len(g1),len(group1)):
            g1.append(small)
        for i in range(len(g2),len(group2)):
            g2.append(small)
        for i,g in enumerate(g1):
            if g < small:
                g1[i] = small
#             g1[i] = np.log(g1[i])
        for i,g in enumerate(g2):
            if g < small:
                g2[i] = small
#             g2[i] = np.log(g2[i])


        t,p = ttest_ind(g1,g2,equal_var=True)
        test_results.append((ps,t,p,g1,g2))
        
test_results = sorted(test_results,key = lambda x: x[2])
for ps,t,p,g1,g2 in test_results[:]:
#     if np.floor(ps.mean_mz) == 84:
        print ps.mean_mz,p
#         g1peaks = []
#         g2peaks = []
#         for p in ps.peaks:
#             if p[2] in group1:
#                 g1peaks.append(p)
#             else:
#                 g2peaks.append(p)
#         print "Group1:"
#         for p in g1peaks:
#             print '\t',p[1],p[2]
#         print
#         print "Group2:"
#         for p in g2peaks:
#             print '\t',p[1],p[2]
#         print
        print
        for g in g1:
            print g
        print
        for g in g2:
            print g
        print
        print
        print
        print
        print
        

9858 peaks
279.159218521 1.5402352575e-05

1906508.0
1891911.375
1697109.25
1666368.625
1629545.75
1444555.875
1781327.375
1539271.125
2294362.0
1826642.25
1302106.125
1144857.625

1198114.0
994032.3125
931715.1875
739966.625
738229.875
539306.125
1142264.625
1122390.5
0.001





330.336942365 3.58037274971e-05

166504.8125
289085.6875
328971.15625
174022.265625
214196.40625
338783.09375
115149.515625
333131.84375
229202.21875
308182.625
180086.46875
110471.8125

126920.664062
119431.742188
131096.765625
90422.96875
0.001
0.001
0.001
0.001
0.001





210.170144183 6.85001728097e-05

251479.21875
366620.625
185896.265625
244350.984375
462090.96875
299717.4375
300844.59375
430789.21875
233418.375
187528.234375
225634.09375
185580.8125

161959.1875
136849.25
140979.9375
223890.765625
0.001
0.001
0.001
0.001
0.001





227.164318293 0.000143057623566

231681.125
273974.875
371714.75
211360.296875
257689.359375
175245.375
184330.15625
236066.78125
219220.5625
231344.890625
183260.515625
0.0

311.065191286 0.0390678916739

290502.9375
47290.7539062
254327.546875
779088.0
464870.5625
186234.9375
630698.75
0.001
0.001
0.001
0.001
0.001

1783326.875
1809600.625
1618768.75
1123113.5
310627.9375
201027.03125
75984.453125
0.001
0.001





264.038392814 0.03914545611

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

81784.8359375
121862.164062
65095.3359375
0.001
0.001
0.001
0.001
0.001
0.001





314.932200736 0.0393279497708

139262.40625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

159490.609375
151897.953125
206969.90625
185406.75
0.001
0.001
0.001
0.001
0.001





245.000029596 0.0394977820233

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

408465.25
383969.34375
205802.0
0.001
0.001
0.001
0.001
0.001
0.001





145.122511191 0.0395788162031

2895113.25
3551142.0
2454832.75
2239217.75
3421458.25
3227575.25
3582666.0
3155911.75
3204612.75
3101006.0
3538112.75
3908462.5

2401062.25
2015071.5
1894184.625
139

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

194437.578125
230316.578125
340109.1875
0.001
0.001
0.001
0.001
0.001
0.001





296.962966365 0.115148693595

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

53516.796875
105634.9375
0.001
0.001
0.001
0.001
0.001
0.001
0.001





529.075571803 0.115373092456

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

120420.125
60764.5703125
0.001
0.001
0.001
0.001
0.001
0.001
0.001





233.021334146 0.1155679148

309897.9375
143484.84375
523458.5625
702957.875
1723564.125
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





185.153848779 0.117055974927

291131.09375
283253.8125
253962.84375
317156.21875
262277.03125
290586.6875
308200.71875
259614.734375
287511.0625
286807.28125
0.001
0.001

246359.703125
292625.0
310217.34375
205755.015625
258871.3125
0.001
0.001
0.001
0.001





167.105770849 0.11705984624

259871.46875
241547.5

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





315.004115395 0.2175314548

260615.734375
277164.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





433.092912104 0.217593386557

173717.046875
162174.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





71.95212876 0.217594966671

386499.0625
248175.546875
139966.015625
1406691.0
492726.375
352163.65625
480841.65625
411651.6875
343407.84375
351609.0
794890.75
671238.8125

476693.03125
561569.375
480649.5
513846.8125
526248.3125
493434.875
960989.1875
1025627.375
11394235.0





275.277495258 0.217671910071

30466.1953125
32902.0742188
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





120.024092106 0.217681857616

211307.4375
228424.4375
0.001
0.001
0.001
0.001
0.001
0.

170481.5625
666449.875
621582.1875
655961.3125
1148405.25
453016.21875
0.001





365.2777808 0.299513079277

182579.953125
729537.5
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





258.101552166 0.299585264395

151690.515625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

137714.265625
286041.34375
0.001
0.001
0.001
0.001
0.001
0.001
0.001





198.05281982 0.300118668171

141311.8125
688933.9375
1077225.875
2375672.0
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

87956.0703125
353531.59375
398530.03125
0.001
0.001
0.001
0.001
0.001
0.001





544.363671269 0.300281059338

149397.765625
38733.8203125
136360.03125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

57850.8515625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





262.055874401 0.300654303668

402424.9375
99200.5703125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

519495.5
710812.125
785537.9375
921410.4375
669989.8125
208574.234375
0.001





193.002111761 0.52732693171

181966.8125
430340.8125
41951.8828125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

323511.84375
161291.171875
246656.921875
85159.265625
0.001
0.001
0.001
0.001
0.001





339.102971954 0.531150125071

35295.8007812
92041.03125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

38374.9179688
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





146.09262598 0.531326088154

271831.65625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

198486.8125
219361.78125
0.001
0.001
0.001
0.001
0.001
0.001
0.001





237.028227866 0.533229306777

177653.953125
582855.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

1401557.5
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





121.958145655 0.538430395564

157762.03125
86494.5625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

435939.25
0.001
0.001
0.001
0.001
0.001
0.001
0.0

Write out a csv file for processing

In [80]:
for log in ['log','nolog']:
    for missing in ['nan',0.01]:
        for normalise in ['none','tus']:
            options = {'log':log,'missing':missing,'normalise':normalise}
            

            if options['normalise'] == 'tus':
                normalised_data = {}
                for f,spec in data.items():
                    mz_list,i_list = zip(*spec)
                    tot_intensity = sum(i_list)
                    new_intensity = [1000.0*i/tot_intensity for i in i_list]
                    new_spec = zip(mz_list,new_intensity)
                    normalised_data[f] = new_spec
            elif options['normalise'] == 'none':
                normalised_data = data
            
            g = Greedy()
            mp = g.process(normalised_data,mz_tol)
                
            display_names = {}
            for g in group1+group2:
                display_names[g] = g.split('/')[-1]
            import csv
            all_names = list(group1+group2)
            res = []


            heads = ["mz"] + [display_names[a] for a in all_names] + ["pval"]
            for ps in mp:
                row = [ps.mean_mz]
                g1 = []
                g2 = []
                for n in all_names:
                    peak = filter(lambda x: x[2] == n,ps.peaks)
                    if len(peak) == 0:
                        row.append("nan")
                    else:
                        row.append(peak[0][1])
                    if n in group1:
                        g1.append(row[-1])
                    if n in group2:
                        g2.append(row[-1])
                if options['missing'] == 'nan':
                    g1 = [a for a in g1 if not a == "nan"]
                    g2 = [a for a in g2 if not a == "nan"]
                else:
                    g1 = [a if not a == "nan" else options["missing"] for a in g1]
                    g2 = [a if not a == "nan" else options["missing"] for a in g2]
                if options['log'] == 'log':
                    g1 = [np.log10(a) for a in g1]
                    g2 = [np.log10(a) for a in g2]
                if len(g1) > 1 and len(g2) > 1:
                    t,p = ttest_ind(g1,g2,equal_var=True)
                else:
                    p = 1
                row.append(p)
                res.append(row)

            res = sorted(res,key = lambda x: x[-1])


            with open("burns_{}_{}_{}.csv".format(options['log'],options['missing'],options['normalise']),'w') as f:
                writer = csv.writer(f)
                writer.writerow(heads)
                for r in res:
                    writer.writerow(r)

9858 peaks
9858 peaks
9858 peaks
9858 peaks
9858 peaks
9858 peaks
9858 peaks
9858 peaks


In [71]:
for r in res[:10]:
    print r[0],r[-1]

279.159218521 1.35563390372e-05
462.146843969 1.73407167063e-05
305.251726956 2.9348596183e-05
304.248307105 5.55628498956e-05
356.279807222 0.000268902663636
280.162638745 0.000361379530556
295.070290712 0.000570483562544
158.154128944 0.000677048047452
143.106848706 0.000924332932252
288.225168757 0.00115745746591
